# 🚀 YouTube Sermon Ingestion

This notebook ingests YouTube sermons into the **production database** via the API.

### Steps:
1. Run Cell 1 to install dependencies
2. Run Cell 2 to upload `cookies.txt` (optional, prevents IP blocks)
3. **Edit Cell 3** — paste your `CLERK_TOKEN` and set the `CHANNEL_URL`
4. Run Cell 3 to start ingestion

### How to get your CLERK_TOKEN:
1. Open [grii.app](https://grii.app) in your browser
2. Open DevTools (F12) → Network tab
3. Send any chat message
4. Find the `stream` request → copy the `Authorization: Bearer <token>` value
5. Paste the token (without `Bearer `) into `CLERK_TOKEN` below

In [ ]:
!pip install -q yt-dlp requests

In [ ]:
# (Optional) Upload cookies.txt to prevent YouTube IP blocks
import os
if not os.path.exists('cookies.txt'):
    print('Upload cookies.txt if you have one (optional).')
    try:
        from google.colab import files
        uploaded = files.upload()
        print(f'Uploaded: {list(uploaded.keys())}')
    except Exception:
        print('No file uploaded. Continuing without cookies.')
else:
    print('cookies.txt already exists.')

In [ ]:
import json
import os
import time
import requests
import yt_dlp

# ============================================================
# CONFIGURATION — edit these before running
# ============================================================

API_BASE_URL = 'https://sermon-rag-production.up.railway.app'
CLERK_TOKEN = ''  # <-- PASTE YOUR TOKEN HERE
CHANNEL_URL = 'https://www.youtube.com/@Reformed21TV/videos'

SKIP_KEYWORDS = [
    'Sermon Clips', 'Sermon Clip', 'Koor ', 'Virtual Ensemble',
    'Virtual Choir', 'Sekolah Minggu', 'Jendela Anak', 'Perkenalan',
    'Thoughts from His Servants', 'Cuplikan', 'Highlight',
]

BATCH_SIZE = 3
BATCH_DELAY = 10
LANGUAGE = 'id'
COOKIES_FILE = 'cookies.txt'

# ============================================================

def get_full_sermon_urls():
    ydl_opts = {'quiet': True, 'no_warnings': True, 'extract_flat': True, 'skip_download': True}
    if os.path.exists(COOKIES_FILE):
        ydl_opts['cookiefile'] = COOKIES_FILE
        print(f'🍪 Using cookies from {COOKIES_FILE}')
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(CHANNEL_URL, download=False)
        entries = info.get('entries', [])
    sermons = []
    for e in entries:
        title = e.get('title', '')
        if any(kw.lower() in title.lower() for kw in SKIP_KEYWORDS):
            continue
        sermons.append({'id': e['id'], 'title': title, 'url': f'https://www.youtube.com/watch?v={e["id"]}'})
    return sermons


def ingest_batch(urls, token):
    resp = requests.post(
        f'{API_BASE_URL}/api/ingest/youtube',
        json={'urls': urls, 'language': LANGUAGE},
        headers={'Authorization': f'Bearer {token}'},
        timeout=300,
    )
    resp.raise_for_status()
    return resp.json()


# ============================================================
# MAIN
# ============================================================

if not CLERK_TOKEN:
    print('❌ Set CLERK_TOKEN above before running!')
else:
    print('📺 Fetching channel video list...')
    sermons = get_full_sermon_urls()
    print(f'Found {len(sermons)} full sermons to ingest.\n')

    total_processed = 0
    total_chunks = 0
    total_errors = []
    total_skipped = 0

    for i in range(0, len(sermons), BATCH_SIZE):
        batch = sermons[i : i + BATCH_SIZE]
        batch_urls = [s['url'] for s in batch]
        batch_num = (i // BATCH_SIZE) + 1
        total_batches = (len(sermons) + BATCH_SIZE - 1) // BATCH_SIZE

        print(f'--- Batch {batch_num}/{total_batches} ---')
        for s in batch:
            print(f'  📹 {s["title"]}')

        try:
            start = time.time()
            result = ingest_batch(batch_urls, CLERK_TOKEN)
            elapsed = time.time() - start

            processed = result.get('sources_processed', 0)
            chunks = result.get('chunks_created', 0)
            errors = result.get('errors', [])
            skipped = len(batch) - processed - len(errors)

            total_processed += processed
            total_chunks += chunks
            total_errors.extend(errors)
            total_skipped += skipped

            print(f'  ✅ {processed} ingested, {skipped} skipped, {len(errors)} errors, {elapsed:.1f}s')
            if errors:
                for err in errors:
                    print(f'    ⚠️ {err}')
        except requests.exceptions.HTTPError as e:
            print(f'  ❌ API Error: {e.response.status_code} — {e.response.text[:200]}')
            if e.response.status_code in (401, 403):
                print('     Token expired or not admin. Get a fresh token!')
                break
            total_errors.append(str(e))
        except Exception as e:
            print(f'  ❌ Error: {e}')
            total_errors.append(str(e))

        print(f'  📊 Total: {total_processed} ingested, {total_skipped} skipped, {total_chunks} chunks')

        if i + BATCH_SIZE < len(sermons):
            print(f'  ⏱️ Waiting {BATCH_DELAY}s...')
            time.sleep(BATCH_DELAY)

    print(f'\n{"=" * 60}')
    print(f'🎉 DONE! {total_processed} ingested, {total_skipped} skipped, {total_chunks} chunks, {len(total_errors)} errors')
    print(f'{"=" * 60}')